# News Knowledge Graph

In [1]:
import requests
from IPython.core.display import HTML

In [33]:
collection="news"
query="wetter"

request = {
    "query": query,
    "params": {
        "df": ['title', 'text'],
        "fore": "{!type=$defType qf=$qf v=$q}",
        "back": "*:*",
        "defType": "edismax",
        "rows": 0,
        "echoParams": "none",
        "omitHeader": "true"
    },
    "facet": {
        "facet_field": {
            "type": "terms",
            "field": "facet_field",
            "sort": { "relatedness": "desc"},
            "mincount": 1,
            "limit": 20,
            "facet": {
                "relatedness": {
                    "type": "func",
                    "func": "relatedness($fore,$back)"
                    #"min_popularity": 0.00000005
                }
            }            
        }
    }
}
request

{'query': 'wetter',
 'params': {'df': ['title', 'text'],
  'fore': '{!type=$defType qf=$qf v=$q}',
  'back': '*:*',
  'defType': 'edismax',
  'rows': 0,
  'echoParams': 'none',
  'omitHeader': 'true'},
 'facet': {'facet_field': {'type': 'terms',
   'field': 'facet_field',
   'sort': {'relatedness': 'desc'},
   'mincount': 1,
   'limit': 20,
   'facet': {'relatedness': {'type': 'func',
     'func': 'relatedness($fore,$back)'}}}}}

In [34]:
solr_url = 'http://localhost:8983/solr/'
response = requests.post(f"{solr_url}{collection}/select", json=request).json()


In [35]:
response

{'response': {'numFound': 67, 'start': 0, 'numFoundExact': True, 'docs': []},
 'facets': {'count': 67,
  'facet_field': {'buckets': [{'val': 'wetter',
     'count': 67,
     'relatedness': {'relatedness': 0.62119,
      'foreground_popularity': 0.00687,
      'background_popularity': 0.00862}},
    {'val': 'grad',
     'count': 61,
     'relatedness': {'relatedness': 0.61713,
      'foreground_popularity': 0.00626,
      'background_popularity': 0.00728}},
    {'val': 'bewoelkt',
     'count': 36,
     'relatedness': {'relatedness': 0.53534,
      'foreground_popularity': 0.00369,
      'background_popularity': 0.00369}},
    {'val': 'regen',
     'count': 46,
     'relatedness': {'relatedness': 0.53433,
      'foreground_popularity': 0.00472,
      'background_popularity': 0.00605}},
    {'val': 'aussichten',
     'count': 43,
     'relatedness': {'relatedness': 0.51586,
      'foreground_popularity': 0.00441,
      'background_popularity': 0.00585}},
    {'val': 'trocken',
     'coun

In [36]:
html_str = '<h3>Anzahl Treffer: ' + str(response['response']['numFound']) + '</h3>'
html_str += '<h3>Facetten</h3>'

for bucket in response["facets"]["facet_field"]["buckets"]:
    # if len(bucket["val"]) < 100:
    html_str += "<b>" + bucket["val"] + ": " + str(bucket["count"]) + "</b> (" + str(bucket["relatedness"]["relatedness"]) + ')<br>'

HTML(html_str)

In [38]:
request = {
    "query": "'wetter^0.62 grad^0.61 bewoelkt^0.53 regen^0.53'",
    "params": {
        "df": ['text'],
        "fore": "{!type=$defType qf=$qf v=$q}",
        "back": "*:*",
        "defType": "edismax",
        "mm": 2,        
        "rows": 10,
        "qf": "text",
        "echoParams": "none",
        "omitHeader": "true"
    }
}
# request

In [39]:
solr_url = 'http://localhost:8983/solr/'
response = requests.post(f"{solr_url}{collection}/select", json=request).json()
# response

In [40]:
html_str = '<h3>Anzahl Treffer: ' + str(response['response']['numFound']) + '</h3>'
html_str += '<h3>Dokumente</h3>'

docs = response["response"]["docs"]

for doc in docs:
    for key in doc.keys():
        value = doc[key]
        html_str += '<br><b>' + key + '</b><br>'
        if isinstance(value,list):
            for elt in value:
                html_str += (elt.replace('\\n', '').replace(", '", "'"))
        else:
            html_str += (value.replace('\\n', '').replace(", '", "'"))
    html_str += '<hr/>'
HTML(html_str)